In [1]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --to notebook --inplace q1.ipynb
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --to notebook --inplace q2.ipynb


[NbConvertApp] WARNING | pattern 'q1.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Path to your dataset
path = "/content/drive/MyDrive/AIRL_Internship_assignment"
os.chdir(path)

# Check files inside the folder
print("Files inside folder:", os.listdir(path))


Files inside folder: ['cifar-10-python.tar.gz', 'q1.ipynb', 'q2.ipynb']


In [ ]:
import tarfile

# Path to tar.gz file
tar_path = "/content/drive/MyDrive/AIRL_Internship_assignment/cifar-10-python.tar.gz"

# Extract into /content/cifar10 folder
with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path="/content/cifar10")

print("Extracted files:", os.listdir("/content/cifar10"))


/tmp/ipython-input-718806688.py:8: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="/content/cifar10")


Extracted files: ['cifar-10-batches-py']


In [ ]:
import tarfile

# Path to tar.gz file
tar_path = "/content/drive/MyDrive/AIRL_Internship_assignment/cifar-10-python.tar.gz"

# Extract into /content/cifar10 folder
with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path="/content/cifar10")

print("Extracted files:", os.listdir("/content/cifar10"))


/tmp/ipython-input-718806688.py:8: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="/content/cifar10")


Extracted files: ['cifar-10-batches-py']


In [ ]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 using extracted path
trainset = torchvision.datasets.CIFAR10(
    root="/content/cifar10",
    train=True,
    download=False,   # since you already extracted
    transform=transform
)

testset = torchvision.datasets.CIFAR10(
    root="/content/cifar10",
    train=False,
    download=False,
    transform=transform
)

print("Train set size:", len(trainset))
print("Test set size:", len(testset))


Train set size: 50000
Test set size: 10000


In [ ]:
# Q1: Vision Transformer on CIFAR-10 (Final Optimized Code)

# Install required libraries
!pip install timm torch torchvision -q

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import timm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Transform for CIFAR-10 (resize for ViT input 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 dataset (from extracted folder)
trainset = torchvision.datasets.CIFAR10(
    root="/content/cifar10", train=True, download=False, transform=transform
)
testset = torchvision.datasets.CIFAR10(
    root="/content/cifar10", train=False, download=False, transform=transform
)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

# Vision Transformer (tiny for fast demo, pretrained on ImageNet)
model = timm.create_model(
    'vit_tiny_patch16_224', pretrained=True, num_classes=10
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

# Training loop (2 epochs only for demo)
for epoch in range(2):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for imgs, labels in trainloader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/2], Loss: {total_loss/len(trainloader):.4f}, Train Acc: {acc:.2f}%")

# Evaluation
correct, total = 0, 0
model.eval()
with torch.no_grad():
    for imgs, labels in testloader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"✅ Final Test Accuracy (ViT-Tiny, 2 epochs): {100*correct/total:.2f}%")


Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

Epoch [1/2], Loss: 0.2637, Train Acc: 90.99%
Epoch [2/2], Loss: 0.1368, Train Acc: 95.39%
✅ Final Test Accuracy (ViT-Tiny, 2 epochs): 92.80%
